# Seventh GF Summer School 2021

----

<img align="left" src="resources/accelerated-text-logo.png" width="400">

Accelerated Text is the natural language generation engine. It will automatically create narratives highlighting outliers, describing trends, and providing explanations in the style and wording most suitable for the intended readers.

### Why low code?

* Closed NLG systems can not react to changing requirements or previously unobserved facts.
* Someone working in a given domain has to be able change the logic defining how data is transformed into narrative.
* Having a domain specific abstraction above the grammar explains how text was produced given a data set. 

### Start the system

Than start the application

```
make run-app
```

Detailed documentation at https://accelerated-text.readthedocs.io/en/latest/installation/

### Elements of Accelerated Text

* Visual programming language (Blockly)
* Main abstractions:
    * Document Plan
    * Dictionary
    * Abstract Meaning representation
    * Grammatical Framework
* Data Enrichment


## The simpliest document plan

Construct a very simple and not very useful plan by connecting words.

<img align="left" src="resources/house_on_the_hill_1_dp.png">

### Connect with the Notebook

Accelerated Text exposes API that allows:
* load existing document plans
* save document plans
* connect data sources
* get text generation results

Full API description can be found here:
- https://accelerated-text.readthedocs.io/en/latest/nlg-api/
- https://accelerated-text.readthedocs.io/en/latest/graphql/

In [1]:
from acctext import AcceleratedText

at = AcceleratedText()
at.health()

{'health': 'Ok'}

Load previously constructed document plans

In [2]:
at.clear_state()

In [3]:
at.restore_state('resources/state.zip')

### Abstract Meaning Representation and Domain Grammar

<img align="left" src="resources/house_on_the_hill_amr.png" width="200">


**AMR** http://localhost:8080/amr 

Define high level domain specific blocks which can be used by non-linguists.


**DLG** http://localhost:8080/dlg 

RGL level where grammar developers work.

### Semantic Graph

<img align="left" src="resources/house_on_the_hill_1.png" width="200">

### Compex Document Plan and its Semantic Graph

Semantic Graph for _Compex plan_



<img align="left" src="resources/complex_plan.png">

## Change plan and interact with AT

Changing document plan can be done via modifications to:
* dictionary
* document plan
* AMR or Grammar

Let us say that in the house I live not just with _borther_ but with the _sister_ as well.

First we need to expand the dictionary.


In [11]:
at.create_dictionary_item('sister', 'N', ['sister'])

{'id': 'sister_N_Eng',
 'key': 'sister',
 'forms': ['sister'],
 'category': 'N',
 'language': 'Eng',
 'attributes': {}}

## Working with data files

Adding the file can be done via AT user interface or directly via API:

In [7]:
at.create_data_file('test_data.csv', ['A', 'B', 'C'], [[1, 2, 3], [11, 22, 33]])

{'id': 'test_data.csv'}

## Adding IF and Variable blocks

<img align="left" src="resources/complex_plan_if_var.png" width="350">

TODO some text here

## Grammar automation

<img align="left" src="resources/syntax_graph.png" width="450">

### Auto-transform to correct type

- For example, we have an AMR with parameter of NP
- We attach dictionary item of type N
- Transformation takes place, that makes the argument compatible: (mkNP (mkCN x))

### Modifier selection

If we have a two argument operation “Pron -> NP -> NP”, and there are no ambiguities, we can use modifier block to construct this operation (“he” + “house” = “his house”)



## Get generation results

In [8]:
result = at.generate('Complex Plan', data={"A": "CCCC"})
result['variants']

['A house was on the CCCC where he and his brother lived. It was called the old house.']

You can get results for multiple data rows

In [9]:
results = at.generate_bulk('Complex Plan', data=[{"A": "CCC"}, 
                                                 {"A": "AAA"}])
[x['variants'] for x in results]

[['A house was on the CCC where he and his brother lived. It was called the old house.'],
 ['A house was on the AAA where he and his brother lived. It was called the old house.']]

Export State

In [10]:
at.export_state('resources/state.zip')

## Prototype based grammar creation

* Parse non trivial sentence
* Show how it can be abstracted. Going from a single example sentence to more possibilities.
* Go back to JN to emphasise this part - ease of use for the end user - close the presentation.


Generate grammar `make build-grammar`

```clojure
[{:name        "Located"
  :description "Object_N -> Location_N -> Clause"
  :category    "Cl"
  :frames      ["house on the hill"]
  :arguments   [{:name "Object" :category "N" :key "house"}
                {:name "Location" :category "N" :key "hill"}]}]
```

## Building blocks outside GF

- data enrichment (Wikifunctions or our)
- ability to add any "text/data to text" functions

## Further development


* Parser fails to parse large section of sentences
* Difficulty in naming things (like AMRs and their parameters). Joining FrameNet and AMR.
* Building reusable AMR

## Last but not least - it is all built with functional language

```clojure
(defn generate-text [semantic-graph context lang]
  (map (fn [{:keys [text]}]
         {:text     text
          :language lang
          :readers  (:readers context)
          :tokens   (nlp/annotate text)})
       (-> semantic-graph
           (grammar/build-grammar (build-context context lang))
           (gf-service/request lang))))
```